In [9]:
# 📌 1. Import Library
import requests
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import base64

In [10]:
# 📌 2. Baca Data dari CSV
csv_path = "data/anxiety_attack_dataset.csv"
data = pd.read_csv(csv_path)

In [11]:
# 📌 3. Ambil 5 Data Secara Acak untuk Uji
sample_data = data.sample(n=1, random_state=42)  # Pilih 5 baris acak untuk diuji

In [12]:
sample_data

,Age,Gender,Occupation,Sleep Hours,Physical Activity (hrs/week),Caffeine Intake (mg/day),Alcohol Consumption (drinks/week),Smoking,Family History of Anxiety,Stress Level (1-10),Heart Rate (bpm during attack),Breathing Rate (breaths/min),Sweating Level (1-5),Dizziness,Medication,Therapy Sessions (per month),Recent Major Life Event,Diet Quality (1-10),Anxiety Category Encoded
1935,58,Female,Teacher,3.3,8.5,47,10,No,No,1,120,19,3,No,No,4,No,7,0


In [13]:
# 📌 4. Fungsi untuk Encode Data ke TFRecord
def encode_example(row):
    # Mengubah satu baris menjadi TFRecord example
    example = tf.train.Example(features=tf.train.Features(feature={
        "Age": tf.train.Feature(int64_list=tf.train.Int64List(value=[row['Age']])),
        "Gender": tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['Gender'].encode()])),
        "Occupation": tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['Occupation'].encode()])),
        "Sleep Hours": tf.train.Feature(float_list=tf.train.FloatList(value=[row['Sleep Hours']])),
        "Physical Activity (hrs/week)": tf.train.Feature(float_list=tf.train.FloatList(value=[row['Physical Activity (hrs/week)']])),
        "Caffeine Intake (mg/day)": tf.train.Feature(int64_list=tf.train.Int64List(value=[row['Caffeine Intake (mg/day)']])),
        "Alcohol Consumption (drinks/week)": tf.train.Feature(int64_list=tf.train.Int64List(value=[row['Alcohol Consumption (drinks/week)']])),
        "Smoking": tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['Smoking'].encode()])),
        "Family History of Anxiety": tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['Family History of Anxiety'].encode()])),
        "Stress Level (1-10)": tf.train.Feature(int64_list=tf.train.Int64List(value=[row['Stress Level (1-10)']])),
        "Heart Rate (bpm during attack)": tf.train.Feature(int64_list=tf.train.Int64List(value=[row['Heart Rate (bpm during attack)']])),
        "Breathing Rate (breaths/min)": tf.train.Feature(int64_list=tf.train.Int64List(value=[row['Breathing Rate (breaths/min)']])),
        "Sweating Level (1-5)": tf.train.Feature(int64_list=tf.train.Int64List(value=[row['Sweating Level (1-5)']])),
        "Dizziness": tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['Dizziness'].encode()])),
        "Medication": tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['Medication'].encode()])),
        "Therapy Sessions (per month)": tf.train.Feature(int64_list=tf.train.Int64List(value=[row['Therapy Sessions (per month)']])),
        "Recent Major Life Event": tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['Recent Major Life Event'].encode()])),
        "Diet Quality (1-10)": tf.train.Feature(int64_list=tf.train.Int64List(value=[row['Diet Quality (1-10)']]))
    }))
    return example.SerializeToString()

In [14]:
# 📌 **5. Encode Banyak Data**
def encode_batch_data(df):
    # Mengonversi beberapa baris data menjadi batch TFRecord
    encoded_data = []
    for _, row in df.iterrows():
        encoded_data.append(encode_example(row))
    return encoded_data

# Encode data batch
encoded_batch = encode_batch_data(sample_data)

# Convert ke base64
encoded_batch_b64 = [base64.b64encode(example).decode('utf-8') for example in encoded_batch]

In [15]:
# 📌 **6. Kirim Request ke Railway dengan Batch Data**
url = "https://ml-pipeline-anxiety-attack-production.up.railway.app/v1/models/anxiety-model:predict"
headers = {"content-type": "application/json"}
data = json.dumps({
    "signature_name": "serving_default",
    "instances": [{"examples": {"b64": example}} for example in encoded_batch_b64]  
})

In [16]:
# 📌 **7. Menangani Respon**
response = requests.post(url, data=data, headers=headers)

# Cek jika response berhasil
if response.status_code == 200:
    try:
        result = response.json().get("predictions")
        if result:
            for idx, prediction in enumerate(result):
                # Ambil nilai probabilitas tertinggi untuk setiap baris
                max_prob = max(prediction)
                max_index = prediction.index(max_prob)  # Dapatkan index dari probabilitas tertinggi
                
                # Mapping nilai prediksi ke label yang sesuai
                if max_index == 0:
                    label = "Mild Anxiety"
                elif max_index == 1:
                    label = "Moderate Anxiety"
                elif max_index == 2:
                    label = "Panic Level Anxiety"
                elif max_index == 3:
                    label = "Severe Anxiety"
                else:
                    label = "Error: Unrecognized prediction value."
                
                # Menampilkan hasil untuk setiap baris
                print(f"Prediction for Row {idx+1}: {label} (Probability: {prediction})")
        else:
            print("Error: No predictions found in the response.")
    except KeyError:
        print("Error: Key 'predictions' not found in the response.")
else:
    print(f"Error: {response.status_code}, {response.text}")

Prediction for Row 1: Moderate Anxiety (Probability: [0.287046373, 0.302494973, 0.119338371, 0.291120201])
